In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

In [3]:
# ÈÄâÊã©Ê®°ÂûãÔºöDistilBERT
MODEL_NAME = 'distilbert-base-uncased' 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Âä†ËΩΩ SQuAD v2 Êï∞ÊçÆÈõÜ
squad_dataset = load_dataset('squad_v2', cache_dir='./data/squad', trust_remote_code=True)

In [5]:
squad_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [6]:
for i in range(11):
    print(squad_dataset['train']['question'][i], squad_dataset['train']['answers'][i])

When did Beyonce start becoming popular? {'text': ['in the late 1990s'], 'answer_start': [269]}
What areas did Beyonce compete in when she was growing up? {'text': ['singing and dancing'], 'answer_start': [207]}
When did Beyonce leave Destiny's Child and become a solo singer? {'text': ['2003'], 'answer_start': [526]}
In what city and state did Beyonce  grow up?  {'text': ['Houston, Texas'], 'answer_start': [166]}
In which decade did Beyonce become famous? {'text': ['late 1990s'], 'answer_start': [276]}
In what R&B group was she the lead singer? {'text': ["Destiny's Child"], 'answer_start': [320]}
What album made her a worldwide known artist? {'text': ['Dangerously in Love'], 'answer_start': [505]}
Who managed the Destiny's Child group? {'text': ['Mathew Knowles'], 'answer_start': [360]}
When did Beyonc√© rise to fame? {'text': ['late 1990s'], 'answer_start': [276]}
What role did Beyonc√© have in Destiny's Child? {'text': ['lead singer'], 'answer_start': [290]}
What was the first album 

In [7]:
hotpotqa_dataset = load_dataset('hotpot_qa', 'distractor', 
                                split='train[:10%]', cache_dir='./data/hotpotqa', trust_remote_code=True) # Âè™Âä†ËΩΩÈÉ®ÂàÜÊï∞ÊçÆÔºåÊµãËØïÁî®

In [8]:
hotpotqa_dataset

Dataset({
    features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
    num_rows: 9045
})

In [9]:
for i in range(3):
    print(hotpotqa_dataset['question'][i],'\n',hotpotqa_dataset['answer'][i], '\n',
          hotpotqa_dataset['type'][i], '\n',hotpotqa_dataset['level'][i], '\n',hotpotqa_dataset['supporting_facts'][i],
          '\n',hotpotqa_dataset['context'][i])

Which magazine was started first Arthur's Magazine or First for Women? 
 Arthur's Magazine 
 comparison 
 medium 
 {'title': ["Arthur's Magazine", 'First for Women'], 'sent_id': [0, 0]} 
 {'title': ['Radio City (Indian radio station)', 'History of Albanian football', 'Echosmith', "Women's colleges in the Southern United States", 'First Arthur County Courthouse and Jail', "Arthur's Magazine", '2014‚Äì15 Ukrainian Hockey Championship', 'First for Women', 'Freeway Complex Fire', 'William Rast'], 'sentences': [["Radio City is India's first private FM radio station and was started on 3 July 2001.", ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).', ' It plays Hindi, English and regional songs.', ' It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.', ' Radio City recently forayed into New Media in May 2008 with

In [15]:
def preprocess_squad(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = [c.strip() for c in examples["context"]]
    
    inputs = tokenizer(
        questions,
        contexts,
        truncation="only_second",
        max_length=512,
        stride=128,
        padding="max_length",
        return_offsets_mapping=True,
        return_tensors="pt",
    )

    start_positions = []
    end_positions = []

    for i in range(len(questions)):
        answer = examples["answers"][i]

        # Â¶ÇÊûúËøô‰∏™Ê†∑Êú¨Ê≤°ÊúâÁ≠îÊ°àÔºàSQuAD v2 ÂèØËÉΩÊúâËøôÁßçÊÉÖÂÜµÔºâ
        if len(answer["answer_start"]) == 0 or len(answer["text"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
            continue

        answer_start = answer["answer_start"][0]
        answer_text = answer["text"][0]

        offsets = inputs["offset_mapping"][i]

        # ÊâæÂà∞ÂåÖÂê´Á≠îÊ°àËµ∑Âßã‰ΩçÁΩÆÁöÑ token
        start_pos = end_pos = 0
        for idx, (start, end) in enumerate(offsets):
            if start <= answer_start < end:
                start_pos = idx
            if start < answer_start + len(answer_text) <= end:
                end_pos = idx
                break
        else:
            # Êâæ‰∏çÂà∞Â∞±ÈªòËÆ§Â°´ 0
            start_pos = end_pos = 0

        start_positions.append(start_pos)
        end_positions.append(end_pos)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    inputs.pop("offset_mapping")  # Ê∏ÖÈô§‰∏çÈúÄË¶ÅÁöÑÂ≠óÊÆµ

    return inputs

In [16]:
def preprocess_hotpotqa(examples):
    questions = [q.strip() for q in examples["question"]]
    # HotpotQA ÁöÑ context ÊòØ‰∏Ä‰∏™ÂµåÂ•óÂàóË°®Ôºö[ [title, paragraph], ... ]
    # Êàë‰ª¨ÊãºÊé•ÊâÄÊúâÊÆµËêΩÂÜÖÂÆπ
    contexts = [" ".join([" ".join(c) for c in entry]) for entry in examples["context"]]
    answers = [a.strip() for a in examples["answer"]]

    inputs = tokenizer(
        questions,
        contexts,
        truncation="only_second",
        max_length=512,
        stride=128,
        padding="max_length",
        return_offsets_mapping=True,
        return_tensors="pt",
    )

    start_positions = []
    end_positions = []

    for i in range(len(questions)):
        answer_text = answers[i]
        context = contexts[i]
        offsets = inputs["offset_mapping"][i]

        # ÊâæÂà∞Á≠îÊ°àËµ∑Âßã‰ΩçÁΩÆ
        answer_start = context.find(answer_text)
        if answer_start == -1:
            # Êó†Ê≥ïÂØπÈΩêÁ≠îÊ°àÔºåËÆæ‰∏∫ 0
            start_positions.append(0)
            end_positions.append(0)
            continue

        start_pos = end_pos = 0
        for idx, (start, end) in enumerate(offsets):
            if start <= answer_start < end:
                start_pos = idx
            if start < answer_start + len(answer_text) <= end:
                end_pos = idx
                break
        else:
            start_pos = end_pos = 0

        start_positions.append(start_pos)
        end_positions.append(end_pos)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    inputs.pop("offset_mapping")

    return inputs

In [17]:
# SQuAD v2 Êï∞ÊçÆÈõÜÈ¢ÑÂ§ÑÁêÜ
tokenized_squad_train = squad_dataset["train"].map(preprocess_squad, batched=True, remove_columns=squad_dataset["train"].column_names)
tokenized_squad_val = squad_dataset["validation"].map(preprocess_squad, batched=True, remove_columns=squad_dataset["validation"].column_names)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

In [13]:
tokenized_hotpotqa = hotpotqa_dataset.map(preprocess_hotpotqa, batched=True, remove_columns=hotpotqa_dataset.column_names)

Map:   0%|          | 0/9045 [00:00<?, ? examples/s]

---

In [19]:
from transformers import DefaultDataCollator
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer

In [20]:
data_collator = DefaultDataCollator(return_tensors="pt")

In [21]:
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
training_args = TrainingArguments(
    output_dir="./qa_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True, 
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad_train,
    eval_dataset=tokenized_squad_val,  # Ê≠£Á°Æ‰ΩøÁî®È™åËØÅÈõÜ
    tokenizer=tokenizer,
    data_collator=data_collator
)

C:\miniconda3\envs\data_science\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\sheng\AppData\Local\Temp\ipykernel_59128\307596042.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [24]:
trainer.train()

TrainOutput(global_step=48870, training_loss=1.1685899772355228, metrics={'train_runtime': 6717.8851, 'train_samples_per_second': 58.196, 'train_steps_per_second': 7.275, 'total_flos': 5.107974402921062e+16, 'train_loss': 1.1685899772355228, 'epoch': 3.0})

In [25]:
trainer.save_model("./model_pkl/qa_model_final")
tokenizer.save_pretrained("./model_pkl/qa_model_final")

('./model_pkl/qa_model_final\\tokenizer_config.json',
 './model_pkl/qa_model_final\\special_tokens_map.json',
 './model_pkl/qa_model_final\\vocab.txt',
 './model_pkl/qa_model_final\\added_tokens.json',
 './model_pkl/qa_model_final\\tokenizer.json')

In [28]:
import evaluate
import numpy as np
from tqdm import tqdm

In [30]:
# Âä†ËΩΩÂÆòÊñπËØÑ‰º∞ÊåáÊ†á
metric = evaluate.load("squad_v2")

# Ëé∑ÂèñÈ™åËØÅÊï∞ÊçÆÈõÜ
tokenized_squad_val = squad_dataset["validation"].map(preprocess_squad, batched=True, remove_columns=squad_dataset["validation"].column_names)
eval_dataset = tokenized_squad_val

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [31]:
# ÂæóÂà∞Ê®°ÂûãÈ¢ÑÊµã
def compute_predictions(trainer, dataset, tokenizer):
    raw_preds = trainer.predict(dataset)
    all_start_logits, all_end_logits = raw_preds.predictions

    predictions = []

    for i in tqdm(range(len(dataset))):
        inputs = tokenizer(
            dataset[i]["question"],
            dataset[i]["context"],
            truncation="only_second",
            max_length=512,
            stride=128,
            return_offsets_mapping=True,
            padding="max_length",
            return_tensors="pt",
        )

        offset_mapping = inputs["offset_mapping"][0]
        input_ids = inputs["input_ids"][0]

        start_logits = all_start_logits[i]
        end_logits = all_end_logits[i]

        # ÊâæÂà∞ÊúÄÂ§ßÊ¶ÇÁéáÁöÑËµ∑Ê≠¢‰ΩçÁΩÆ
        start_idx = np.argmax(start_logits)
        end_idx = np.argmax(end_logits)

        if start_idx >= len(offset_mapping) or end_idx >= len(offset_mapping):
            pred_answer = ""
        else:
            start_char = offset_mapping[start_idx][0]
            end_char = offset_mapping[end_idx][1]
            context = dataset[i]["context"]
            pred_answer = context[start_char:end_char]

        predictions.append({
            "id": dataset[i]["id"],
            "prediction_text": pred_answer,
            "no_answer_probability": 0.0 
        })

    return predictions

In [32]:
# ÊûÑÂª∫ references Ê†ºÂºè
def build_references(dataset):
    references = []
    for i in range(len(dataset)):
        answer = dataset[i]["answers"]
        references.append({
            "id": dataset[i]["id"],
            "answers": {
                "answer_start": answer["answer_start"],
                "text": answer["text"],
            }
        })
    return references

In [ ]:
# ÊâßË°åËØÑ‰º∞
predictions = compute_predictions(trainer, eval_dataset, tokenizer)
references = build_references(eval_dataset)

final_score = metric.compute(predictions=predictions, references=references)
print("Evaluation Results:")
print(final_score)